In [ ]:
%matplotlib inline
import numpy as np
import sys
import math
from sklearn.preprocessing import Imputer
import h5py
import matplotlib.pyplot as plt
import pandas as pd
import os
from os import listdir
from os.path import isfile, join
#import feather

In [ ]:
import helperFunc as hf

In [ ]:
from ROOT import gROOT, gDirectory, TFile, TEventList, TCut, TH1F, TH2F, TCanvas, kRed
from root_numpy import tree2array, root2array
%jsroot on

In [ ]:
indir  = "/scratch3/eirikgr/2L0JNtuples/FCLoose-HighPt-21-2-58/statonly/"
outdir = "/scratch3/eirikgr/ML_files/2L0J/"

Convert ROOT ntuples to h5 files.

In [ ]:
susyFeaturesNtup = []
h5files = []
for mc in ["newMC16D"]:#["newMC16A"]:#,"newMC16D","newMC16E"]:
    mypath = indir+mc+"/"
    onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
    nf = 0
    for f in onlyfiles:
        print "Loading file %s" %f
        if not f.endswith(".root"): continue
        tf = TFile(mypath+f)
        keyList = [key.GetName() for key in tf.GetListOfKeys()]
        ntree = 0
        for key in keyList:
            obj = tf.Get(key)
            outPathDir = "%s/%s"%(outdir,mc)
            if not os.path.exists(outPathDir):
                print "Creating path %s"%outPathDir
                os.makedirs(outPathDir)
            filePath = "%s/%s_%s_%s"%(outPathDir,f.replace(".root",""),key,mc)
            if os.path.exists("%s.h5"%filePath):
                os.remove("%s.h5"%filePath)
            if len(susyFeaturesNtup) == 0: 
                susyFeaturesNtup = hf.getListOfBranches(obj,1)
            hf.writeArrays(mypath+f, key, susyFeaturesNtup, filePath,"", 100000)
            ntree += 1
        nf += 1

Loop over the h5 files from above and create pandas which are merged in to on big panda. It is also adding an additional column which is 0 for background samples and 1 for signal samples.

In [ ]:
frames = []
#df = {}
n = 0
outdir += "/newMC16D"
for f in listdir(outdir):
    if n%10 == 0: print "Doing %i/%i" %(n,len(listdir(outdir)))
    #if "OthersWithoutHiggs_Others_central" in f: continue
    #if not "data" in f: continue
    if f.startswith("all"): continue
    if not isfile(join(outdir, f)) or not f.endswith(".h5"): 
        continue
    h5f = outdir+"/"+f
    print h5f
    eventFile = h5py.File(h5f,"r")
    key = eventFile.keys()[0] #"_".join(h5f.split("_")[-3:-1])
    #print key
    df = pd.DataFrame(np.copy(eventFile[key]),columns=eventFile.attrs["var_names"])
    sLength = len(df['DSID'])
    if "C1C1" in h5f or "SlepSlep" in h5f:
        descr = np.chararray(sLength,itemsize=50)
        descr[:] = "_".join(f.split("_")[-6:-2])
        #print "_".join(f.split("_")[-6:-2])
        df["isSignal"] = pd.Series(np.ones(sLength), index=df.index, dtype='int')
        df["MCtype"]   = pd.Series(descr, index=df.index, dtype='string')
    elif "data" in f:
        descr = f.split("_")[0]
    else:
        #print "Background ",f
        descrkey = ""
        for sp in f.split("_"):
            if "central" in sp: break
            descrkey += "%s_" %sp
        descrkey = descrkey[:-1]
        descr = np.chararray(sLength,itemsize=50)
        descr[:] = descrkey
        #print "_".join(f.split("_")[-5:-3])
        df["isSignal"] = pd.Series(np.zeros(sLength), index=df.index,dtype='int')
        df["MCtype"]   = pd.Series(descr, index=df.index, dtype='string')
        #break
    if n == 0: result = df
    else: result = pd.concat([result,df])
    n += 1
    #break
#eventFile_bkg = h5py.File(outdir+"/Singletop_Singletop_central_newMC16A.h5","r")
#df2 = pd.DataFrame(np.copy(eventFile_bkg['Singletop_central']),columns =eventFile_bkg.attrs["var_names"])

#df = pd.concat([df1,df2])
#df = pd.DataFrame(eventFile_bkg.attrs["var_names"],index=data[:,0]),
#test = pd.read_hdf(outdir+"/OthersWithoutHiggs_Others_central_newMC16A.h5",'Others_central'
#print(df.dtypes)

In [ ]:
descr

In [ ]:
print result

In [ ]:
path = outdir+"/all_bkg_sig.h5"
#feather.write_dataframe(result, path)
result.to_hdf(path,key='result', mode='w')

In [ ]:
path = outdir+"/all_data.h5"
#feather.write_dataframe(result, path)
result.to_hdf(path,key='result', mode='w')

In [ ]:
bkgdf = result.loc[result['isSignal'] == 0]
bkgs = np.unique(bkgdf.iloc[:,[-1]].as_matrix())
print bkgs

In [ ]:
# Inspect some (or all...) of the columns using the names printed in the above cell
variable = 'MET' 
df2 = bkgdf.groupby([variable,"MCtype"])[variable].count().unstack('MCtype').fillna(0)
#df2[['abuse','nff']].plot(kind='bar', stacked=True)
#print df.loc[df[variable].idxmax()]
#print df[variable].idxmax()
#df = df.drop(df[variable].idxmax())
#print df[variable].idxmax()
#print df.loc[df[variable].idxmax()]
#print df.loc[df[variable].idxmax()]
#print df.loc[df[variable] < 1000000
#print df[variable].div(1000)
#hist = df.loc[df[variable] < 1000000][variable].div(1000).hist(bins=1000,facecolor='red')
#hist.set_xlabel(variable)
#plt.yscale("log")
#plt.show()

In [ ]:
df2[bkgs].plot(kind='bar', stacked=True)